In [1]:
import os
from urllib.parse import urljoin

import requests
from requests.adapters import HTTPAdapter, Retry
from bs4 import BeautifulSoup, SoupStrainer

## classe Request

In [2]:
class Request:
    def __init__(self, n_retry: int = 10, timeout: float = 5):
        self.retries = Retry(total=n_retry, backoff_factor=1, status_forcelist=[429, 500, 502, 503, 504])
        self.adapter = HTTPAdapter(max_retries=self.retries)
        self.http = requests.Session()
        self.http.mount("https://", self.adapter)

        self.timeout = timeout

    def get(self, url: str, params: dict = None) -> str:
        if params is None:
            params = {}

        resp = self.http.get(url, params=params, timeout=self.timeout, headers={'accept-language':'pt-BR,pt;q=0.5'})
        resp.raise_for_status()

        return resp

In [3]:
my_request = Request()

## SciELO Artigos
### Funções

In [4]:
scielo_url = lambda x: urljoin('https://www.scielo.br', x)

In [5]:
def volumes_por_ano() -> dict[str:list]:
    req = my_request.get(scielo_url('j/rbef/grid'))
    soup = BeautifulSoup(req.text, 'lxml', parse_only=SoupStrainer('tbody'))

    links = {linha.td.text.strip(): [scielo_url(a['href']) for a in linha.find_all('a')] for linha in soup.find_all('tr')}
    return links

In [6]:
def artigos_por_volume(url: str, formato: str = 'html', idioma: str = 'pt') -> list:
    languages = {'pt': 'Português', 'en': 'Inglês', 'es': 'Espanhol'}
    formatos = {'html': '/?lang=', 'pdf': '/?format=pdf&'}

    lang = languages.get(idioma.lower())
    form = formatos.get(formato.lower())

    req = my_request.get(url)

    strainer = SoupStrainer('a', attrs={'title': lang})
    soup = BeautifulSoup(req.text, 'lxml', parse_only=strainer)

    links = []
    for a in soup.findAll('a'):
        if 'abstract' not in a['href'] and form in a['href']:
            links.append(scielo_url(a['href']))

    return links

In [7]:
def html_do_artigo(text: str) -> str:
    strainer = SoupStrainer('div', attrs={'class': 'articleTxt'})
    return BeautifulSoup(text, 'lxml', parse_only=strainer).prettify()

### Código
Salva os artigos da scielo na pasta `artigos_brutos`

In [8]:
volumes = volumes_por_ano()
del volumes['2023']  # remove 2023

try:
    os.mkdir(os.path.join(os.getcwd(), 'artigos_brutos')) # cria a pasta
except FileExistsError:
    print('Pasta existente!')

path = os.path.join(os.getcwd(), 'artigos_brutos')

n = 0  # Numeração dos artigos
for ano, urls in volumes.items():
    for i, link in enumerate(urls, start=1):  # itera nas urls dos volumes
        all_ = artigos_por_volume(link)

        for j, artigo in enumerate(all_, start=1):  # itera nas urls dos artigos contidos no volume
            nome_arquivo = str(n).rjust(4, "0") + '.html'  # Nome do arquivo em função do n. ex: 0123.html
            path_save = os.path.join(path, nome_arquivo)  # local onde salvar os arquivos

            # print do progresso
            print(f'Ano: {ano} | Volume: {i}/{len(urls)} | Artigo: {str(j).rjust(3, "0")}/{str(len(all_)).rjust(3, "0")} | Total: {str(n).rjust(4, "0")}', end='\r')

            # requesição da url do artigo
            reque = my_request.get(artigo)

            # salva o html artigo completo num arquivo
            with open(path_save, 'w', encoding='utf8') as file:
                file.write(html_do_artigo(reque.text))

            n += 1

Pasta existente!
